In [1]:
# battery storage optmisation with Reinforcement Learning
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
from matplotlib import cm
from pickle import dump

# import model architecture from model scripts directory
sys.path.append('./models') 
sys.path.append('../models') 

# import custom classes:
from scripts.da_electricity_price_model import LSTMCNNModel
from scripts.battery_efficiency import BatteryEfficiency
from scripts.battery_environment import Battery
from scripts.models.dqn_vanilla import DQN_Agent
from scripts.models.dqn_double_dueling import DQN_Agent_double_duel

# seed random number for consistency
seed = 100

# declare environment dictionary params
env_settings = {
	'battery_capacity': 20000,	# rated capacity of battery (kWh)
    'battery_energy': 10000,	# rated power of battery (kW)
    'battery_price': 3,			# battery CAPEX (£/kWh)
    'num_actions': 5,			# splits charge/discharge MWs relative to rated power
    'standby_loss': 0.99,		# standby loss for battery when idle
    'num_episodes': 1000,		# number of episodes 
    'train': True,				# Boolean to determine whether train or test state
    'scaler_transform_path': '../battery-optimisation-with-drl/data/processed_data/da_price_scaler.pkl',				
    'train_data_path': '../battery-optimisation-with-drl/data/processed_data/train_data_336hr_in_24hr_out_unshuffled.pkl', # Path to trian data
    'test_data_path': '../battery-optimisation-with-drl/data/processed_data/test_data_336hr_in_24hr_out_unshuffled.pkl',	 # Path to test data
    'torch_model': '../battery-optimisation-with-drl/models/da_price_prediction.pt',	 # relevant to current file dir
    'price_track': 'true' # 'true' or 'forecasted'
}


ModuleNotFoundError: No module named 'scripts'

In [ ]:
from pickle import dump, load
train_data_path = env_settings['train_data_path']

train_load = open(train_data_path, "rb") 
input_prices = load(train_load)

In [ ]:
input_prices['X_train'][0]

array([[0.03813801, 0.        , 0.        , ..., 0.5       , 0.        ,
        1.        ],
       [0.03570175, 0.04347826, 0.        , ..., 0.5       , 0.        ,
        0.        ],
       [0.03090944, 0.08695652, 0.        , ..., 0.5       , 0.        ,
        0.        ],
       ...,
       [0.03856912, 0.91304348, 0.2       , ..., 0.33333333, 0.        ,
        0.        ],
       [0.03370663, 0.95652174, 0.2       , ..., 0.33333333, 0.        ,
        0.        ],
       [0.03345598, 1.        , 0.2       , ..., 0.33333333, 0.        ,
        0.        ]])

In [ ]:
# import pandas as pd
# import numpy as np

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from pickle import dump, load

# # from workalendar.europe import UnitedKingdom
# # cal = UnitedKingdom()


# # create train and test set data
# n2ex_da = pd.read_csv('../battery-optimisation-with-drl/data/N2EX_UK_DA_Auction_Hourly_Prices_pytorch_train.csv', header=0)

# # convert series to datetime
# n2ex_da['utc_timestamp'] = pd.to_datetime(n2ex_da['utc_timestamp'],format='%d/%m/%Y %H:%M')

# start_date = n2ex_da['utc_timestamp'].min()
# end_date = n2ex_da['utc_timestamp'].max()
# start_year = int(n2ex_da['utc_timestamp'].dt.year.min())
# end_year = int(n2ex_da['utc_timestamp'].dt.year.max())

# for year in range(start_year, end_year + 1): 
#     for holiday in cal.holidays(year):
#         #print(type(pd.Timestamp(holiday[0])), 'holiday')
#         # print(start_date.date, 'start_date')
#         print(start_date <=  pd.Timestamp(holiday[0]) <= end_date)
# add holidays boolean indicator
# holidays = set(holiday[0]
#     for year in range(start_year, end_year + 1) 
#     for holiday in cal.holidays(year)
#     if start_date <=  holiday[0] <= end_date)

# df_times['holiday'] = n2ex_da['utc_timestamp'].isin(holidays).astype(int)

In [ ]:
# import pandas as pd
# import numpy as np

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from pickle import dump, load

# from workalendar.europe import UnitedKingdom
# cal = UnitedKingdom()


# # create train and test set data
# n2ex_da = pd.read_csv('../battery-optimisation-with-drl/data/N2EX_UK_DA_Auction_Hourly_Prices_pytorch_train.csv', header=0)

# # convert series to datetime
# n2ex_da['utc_timestamp'] = pd.to_datetime(n2ex_da['utc_timestamp'],format='%d/%m/%Y %H:%M')

# # train / test reference
# data_ref = 'test'

# # save scaler for inverse transform
# if data_ref == "train":

# 	# normalise the price
# 	scaler = MinMaxScaler()
# 	n2ex_da[['Price_(£)']] = scaler.fit_transform(n2ex_da[['Price_(£)']])

# 	with open(f"../battery-optimisation-with-drl/data/processed_data/da_price_scaler.pkl", "wb") as scaler_store:
# 		dump(scaler, scaler_store)

# else: # load scaler

# 	scaler = load(open(f'../battery-optimisation-with-drl/data/processed_data/da_price_scaler.pkl', 'rb'))
# 	n2ex_da[['Price_(£)']] = scaler.fit_transform(n2ex_da[['Price_(£)']])

# # ts_df = pd.concat(days_df)
# ts = n2ex_da['Price_(£)']
# ts = np.expand_dims(ts.values, axis=-1)

# dates = n2ex_da['utc_timestamp'].values
# dates = np.array(dates, dtype = 'datetime64[ns]')

# # data engineering
# df_times = pd.DataFrame()
# df_times['hour'] = n2ex_da['utc_timestamp'].dt.hour
# df_times['day'] = n2ex_da['utc_timestamp'].dt.day
# df_times['month'] = n2ex_da['utc_timestamp'].dt.month 
# df_times['day_of_year'] = n2ex_da['utc_timestamp'].dt.dayofyear 
# df_times['day_of_week'] = n2ex_da['utc_timestamp'].dt.dayofweek 
# # df_times['year'] = n2ex_da.index.year 
# df_times['weekend'] = df_times['day_of_week'].apply(lambda x: 1 if x>=5 else 0)

# start_year = int(n2ex_da['utc_timestamp'].dt.year.min())
# end_year = int(n2ex_da['utc_timestamp'].dt.year.max())

# start_date = n2ex_da['utc_timestamp'].min()
# end_date = n2ex_da['utc_timestamp'].max()

# # add holidays boolean indicator
# holidays = set(holiday[0]
#     for year in range(start_year, end_year + 1) 
#     for holiday in cal.holidays(year)
#     if start_date <=  pd.Timestamp(holiday[0]) <= end_date)

# df_times['holiday'] = n2ex_da['utc_timestamp'].isin(holidays).astype(int)

# # normalise features in each column
# for col_idx, col in enumerate(df_times.columns.values[:-2]): # ignore the last two colums as one-hot encoding
# 	df_times[col] = (df_times[col] - np.min(df_times[col]))  / (np.max(df_times[col]) - np.min(df_times[col]))

# # convert times to array
# times_data = df_times.values

# # group days for input and output train/test data set
# def input_output(ts, times_data, dates, input_seq_size, output_seq_size):
# 	x_input, x_times_data, y_output, in_times, out_times  = [], [], [], [], []
# 	input_start = 0
# 	output_start = input_seq_size

# 	while (output_start + output_seq_size) < len(ts):

# 		x_time = np.empty(((input_seq_size)), dtype = 'datetime64[ns]')
# 		y_time = np.empty(((output_seq_size)), dtype = 'datetime64[ns]')

# 		input_end = input_start + input_seq_size
# 		output_end = output_start + output_seq_size

# 		input_seq = ts[input_start:input_end]
# 		x_input.append(input_seq)
# 		eng_times_data = times_data[input_start:input_end]
# 		x_times_data.append(eng_times_data)
# 		output_seq = ts[output_start:output_end]
# 		y_output.append(output_seq)
	
# 		x_time[:] = dates[input_start:input_end]
# 		in_times.append(x_time)
# 		y_time[:] = dates[output_start:output_end]
# 		out_times.append(y_time)

# 		input_start += 24
# 		output_start += 24

# 		# steps for test data set
# 		# input_start += output_seq_size 
# 		# output_start += output_seq_size

# 	x_input = np.array(x_input)
# 	x_times_data = np.array(x_times_data)
# 	y_output = np.array(y_output)
# 	x_input_times = np.array(in_times, dtype = 'datetime64[ns]')
# 	y_output_times = np.array(out_times, dtype = 'datetime64[ns]')

# 	x_input = np.concatenate([x_input, x_times_data], axis=-1)

# 	X_train, X_test, y_train, y_test = train_test_split(x_input, y_output, test_size=0.99, shuffle=False) # split used to capture 2019 only

# 	X_train_times, X_test_times, y_train_times, y_test_times = train_test_split(x_input_times, y_output_times, test_size=0.99, shuffle=False)

# 	train_data = {
# 		'X_train': X_train,
# 		'y_train': y_train,
# 		'X_train_times': X_train_times,
# 		'y_train_times': y_train_times
# 	}

# 	test_data = {
# 		'X_test': X_test,
# 		'y_test': y_test,
# 		'X_test_times': X_test_times,
# 		'y_test_times': y_test_times
# 	}

# 	print(*[f'{key}: {train_data[key].shape}' for key in train_data.keys()], sep='\n')
# 	print(*[f'{key}: {test_data[key].shape}' for key in test_data.keys()], sep='\n')

# 	return train_data, test_data


# train_data, test_data = input_output(ts[2:], times_data[2:], dates[2:], input_seq_size=168, output_seq_size=24)


# # save data
# with open(f"../battery-optimisation-with-drl/data/processed_data/train_data_336hr_in_24hr_out_unshuffled.pkl", "wb") as trainset:
# 	dump(train_data, trainset)

# with open("../battery-optimisation-with-drl/data/processed_data/test_data_336hr_in_24hr_out_unshuffled.pkl", "wb") as testset:
# 	dump(test_data, testset)








In [ ]:
import sys
print(sys.path[0])

/gpfs/alpine/csc499/scratch/vincehass/battery-optimisation-with-drl


In [ ]:
# DQN models
dqn_models = ['vanilla', 'double_dueling', 'NN']

# store profits for each model in dicitonary
dqn_model_profits = {}

# training loop:
for model in dqn_models: # loop for each model type

	# episode + time range parameters
	n_episodes = 12000 # max 67925
	time_range = 168

	# e-greedy params
	if model != 'NN' and model != 'double_dueling_NN':
		epsilon = 1.0
		epsilon_end = 0.01
		epsilon_decay = 0.9995
	else:
		epsilon = 0
		epsilon_end = 0
		epsilon_decay = 0

	# further DQN params
	learning_rate = 25e-5 
	buffer_size = int(1e5)
	batch_size = 32 # 64 best
	gamma = 0.99
	# tau = 1e-3
	tau = 0.001 # i.e. 1 'hard' update
	update = 16

	# instaniate environment 
	env = Battery(env_settings)
	state_size = (env.observation_space.shape[0])
	action_size = len(env.action_space)

	print(model)
	# instaniate DQN agent
	if model == "double_dueling" or model == "double_dueling_NN":
		dqn_agent = DQN_Agent_double_duel(state_size, action_size, learning_rate, buffer_size, gamma, tau, batch_size, seed, soft_update=True, qnet_type=model)
	else:
		dqn_agent = DQN_Agent(state_size, action_size, learning_rate, buffer_size, gamma, tau, batch_size, seed, soft_update=True, qnet_type=model)        

	total_cumlative_profit = 0
	cumlative_profit = [0]

	# declare arrays to collect info during training
	scores = np.empty((n_episodes)) # list of rewards from each episode
	profits = np.empty((n_episodes))

	for ep in range(n_episodes): # loop through episodes

	    # reset reward and profit vars
		episode_rew = 0
		episode_profit = 0

	    # reset evironment between episodes 
		cur_state = env.reset()

	    # loop through timesteps in each episode
		for step in range(time_range): 

            # action selection
			action = dqn_agent.action(cur_state, epsilon)

	        # env step
			new_state, reward, done, info = env.step(cur_state, action, step)

	        # agent step
			dqn_agent.step(cur_state, action, reward, new_state, update, batch_size, gamma, tau, done)

			cur_state = new_state 
			episode_rew += reward

			# store episode profit 
			episode_profit += info["ts_cost"]
			cumlative_profit.append(cumlative_profit[-1] + info["ts_cost"])

	        # end episode if 'game over'
			if done:
				break

		scores[ep] = episode_rew 
		profits[ep] = episode_profit
		# epsilon = epsilon - (2/(ep+1)) if epsilon > 0.01 else 0.01
		epsilon = max(epsilon*epsilon_decay, epsilon_end)

		print(f"Episode:{ep}\n Reward:{episode_rew}\n Epsilon:{epsilon}\n Profit: {episode_profit}")
    
	dqn_model_profits[model] = cumlative_profit

	# save model + results from trained DQN model 
	torch.save(dqn_agent.qnet.state_dict(),f'/content/drive/My Drive/Battery-RL/trained_models/dqn_{model}.pth')

	with open(f"/content/drive/My Drive/Battery-RL/rewards/rewards_{model}.pkl", "wb") as episode_rewards:
		dump(scores, episode_rewards)

# save timeseries performance df via pickle
with open('/content/drive/My Drive/Battery-RL/rewards/profits.pkl', 'wb') as timeseries_results:
	dump(dqn_model_profits, timeseries_results)

# plot profit comparison
for idx, model in enumerate(dqn_models):
	plt.plot(dqn_model_profits[model], label=f'{model}')

plt.legend(loc="lower right")
plt.show()


/ccs/proj/csc499/vincehass/rapids/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/ccs/proj/csc499/vincehass/rapids/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/ccs/proj/csc499/vincehass/rapids/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


vanilla
_______________________________
Episode:0
 Reward:nan
 Epsilon:0.9995
 Profit: nan
_______________________________
Episode:1
 Reward:nan
 Epsilon:0.9990002500000001
 Profit: nan
_______________________________
Episode:2
 Reward:nan
 Epsilon:0.9985007498750001
 Profit: nan
_______________________________
Episode:3
 Reward:nan
 Epsilon:0.9980014995000627
 Profit: nan
_______________________________
Episode:4
 Reward:nan
 Epsilon:0.9975024987503127
 Profit: nan
_______________________________
Episode:5
 Reward:nan
 Epsilon:0.9970037475009376
 Profit: nan
_______________________________
Episode:6
 Reward:nan
 Epsilon:0.9965052456271871
 Profit: nan
_______________________________
Episode:7
 Reward:nan
 Epsilon:0.9960069930043736
 Profit: nan
_______________________________
Episode:8
 Reward:nan
 Epsilon:0.9955089895078715
 Profit: nan
_______________________________
Episode:9
 Reward:nan
 Epsilon:0.9950112350131176
 Profit: nan
_______________________________
Episode:10
 Reward:nan